In [0]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.models import Model, load_model
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow 
from imutils import paths
from tqdm import tqdm

In [2]:
# from google.colab import drive
# drive.mount('/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


In [0]:
model = load_model('/drive/My Drive/model8')

In [0]:
dict_map = {
    0: 'airport_inside', 1: 'bar', 2: 'bowling', 3: 'buffet', 4: 'casino', 5: 'church_inside', 6: 'cloister',
    7: 'concert_hall', 8: 'elevator', 9: 'fastfood_restaurant', 10: 'florist', 11: 'gameroom', 12: 'gym',
    13: 'hairsalon', 14: 'inside_bus', 15: 'library', 16: 'locker_room', 17: 'movietheater', 18: 'museum', 
    19: 'poolinside', 20: 'prisoncell', 21: 'restaurant', 22: 'subway', 23: 'trainstation', 24: 'waitingroom'
}

In [0]:
def heatmap(orig, intensity=0.5, res=250):

  label = orig.split(os.path.sep)[-2]
  name_image = orig.split(os.path.sep)[-1][:-4]

  img = load_img(orig, target_size=(224, 224))

  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)

  preds = model.predict(x)
  label_predict = dict_map[np.argmax(preds)]

  with tf.GradientTape() as tape:
    last_conv_layer = model.get_layer('Conv_1_bn') # last convolutional layer
    iterate = Model([model.inputs], [model.output, last_conv_layer.output])
    model_out, last_conv_layer = iterate(x)
    class_out = model_out[:, np.argmax(model_out[0])]
    grads = tape.gradient(class_out, last_conv_layer)
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    
  heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer), axis=-1)
  heatmap = np.maximum(heatmap, 0)
  heatmap /= np.max(heatmap)
  heatmap = heatmap.reshape((7, 7))

  img = cv2.imread(orig)

  heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

  heatmap = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)

  img = heatmap * intensity + img

  font = cv2.FONT_HERSHEY_DUPLEX
  text = f'{label_predict}: {str(round(preds.max(),4))}'
  color = (0, 255, 0) if label == label_predict else (0, 0, 255) 

  cv2.putText(img, 
              text, 
              (5, (int) (img.shape[0]*0.97)), 
              font, 
              0.00162162 * img.shape[1],
              color,
              2, 
              cv2.LINE_AA)
  cv2.rectangle(img, (0, 0), (img.shape[1], img.shape[0]), color, 6)

  # cv2_imshow(img)

  filename = '/drive/My Drive/heatmap_new_images/heatmap_%s.png' % (name_image)
  cv2.imwrite(filename,img)

In [0]:
imagePaths = list(paths.list_images('/drive/My Drive/new_images/'))
for imagePath in imagePaths:
  heatmap(imagePath)